# Chapter 3: Fibonacci Example–Part 2
Now, we will generate multiple witnesses to test the soundness of our circuit constraints. Note that we only intend to accept the following set of values for signals "a", "b", and "c". "Soundness" in this context refers to faulty witness successfully verified against the constraints (false positives), so any set of witness assignments that is different from the table below but still passes the constraints incurs a "soundness" error.
| a | b | c |
| - | - | - |
| 1 | 1 | 2 |
| 1 | 2 | 3 |
| 2 | 3 | 5 |
| 3 | 5 | 8 |
| ... | |   |

## Setup
We setup the same circuit and witness in Part 1 which were successfully verified:

In [4]:
from __future__ import annotations
from typing import Tuple

from dsl import Circuit, StepType
from cb import eq
from util import F

class FiboStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

class Fibonacci(Circuit):
    def setup(self):
        self.a = self.forward("a")
        self.b = self.forward("b")

        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        
    def trace(self, args):
        self.add(self.fibo_step, (1, 1))
        a = 1
        b = 2
        for i in range(1, 10):
            self.add(self.fibo_step, (a, b))
            prev_a = a
            a = b
            b += prev_a

fibo = Fibonacci()
fibo_witness = fibo.gen_witness(None)

Now we swap the first step instance from `(1, 1, 2)` to `(0, 2, 2)`. We use the `evil_witness` function to swap step index 0 assignment index 0 to `F(0)` and step index 0 assignment index 0 to `F(2)`.

In [5]:
evil_witness = fibo_witness.evil_witness_test(step_instance_indices=[0, 0], assignment_indices=[0, 1], rhs=[F(0), F(2)])

Print the `evil_witness` to confirm that the swap was successful:

In [6]:
print(evil_witness)

TraceWitness(
	step_instances={
		StepInstance(
			step_type_uuid=103999946909552591427032502489062181386,
			assignments={
				a = 0,
				b = 2,
				c = 2
			},
		),
		StepInstance(
			step_type_uuid=103999946909552591427032502489062181386,
			assignments={
				a = 1,
				b = 2,
				c = 3
			},
		),
		StepInstance(
			step_type_uuid=103999946909552591427032502489062181386,
			assignments={
				a = 2,
				b = 3,
				c = 5
			},
		),
		StepInstance(
			step_type_uuid=103999946909552591427032502489062181386,
			assignments={
				a = 3,
				b = 5,
				c = 8
			},
		),
		StepInstance(
			step_type_uuid=103999946909552591427032502489062181386,
			assignments={
				a = 5,
				b = 8,
				c = 13
			},
		),
		StepInstance(
			step_type_uuid=103999946909552591427032502489062181386,
			assignments={
				a = 8,
				b = 13,
				c = 21
			},
		),
		StepInstance(
			step_type_uuid=103999946909552591427032502489062181386,
			assignments={
				a = 13,
				b = 21,
				c = 34
			},
		),
		StepInstance(
	

Now, generate and verify the proof with `evil_witness`:

In [7]:
fibo.verify_proof(evil_witness)

thread '<unnamed>' panicked at 'index out of bounds: the len is 0 but the index is 0', src/chiquito/src/compiler.rs:651:9
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: index out of bounds: the len is 0 but the index is 0

Surprisingly, `evil_witness` generated a proof that passed verification. This constitutes a soundness error, because the first step instance isn't `(1, 1, 2)` as we initially specified, so why can the witness still pass the constraints?

The answer is simple, because in the first step instance, we never constrained the values of "a" and "b" to 1 and 1 in `setup` of `FiboStep`. We also didn't constrain the first step instance to be `fibo_step`.

You might be wondering: in `trace`, didn't we set "a" and "b" to `(1, 1)` and added `fibo_step` as the first step instance? In fact, `trace` and `wg` are really helper functions for the prover to easily generate a witness, whose data can be tampered with as shown in `evil_witness_test`. The only conditions enforced are defined in circuit and step type `setup`. Therefore, to fix the soundness error, we need to add more constraints, in Part 3.